### Determining the optimal number of hidden layers and neurons for an Artificial Neural Network (ANN) 

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder
from sklearn.pipeline import Pipeline
from scikeras.wrappers import KerasClassifier
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping
import pickle

In [2]:
data=pd.read_csv('Churn_Modelling.csv')
data = data.drop(['RowNumber', 'CustomerId', 'Surname'], axis=1)

label_encoder_gender = LabelEncoder()
data['Gender'] = label_encoder_gender.fit_transform(data['Gender'])

onehot_encoder_geo = OneHotEncoder(handle_unknown='ignore')
geo_encoded = onehot_encoder_geo.fit_transform(data[['Geography']]).toarray()
geo_encoded_df = pd.DataFrame(geo_encoded, columns=onehot_encoder_geo.get_feature_names_out(['Geography']))

data = pd.concat([data.drop('Geography', axis=1), geo_encoded_df], axis=1)

X = data.drop('Exited', axis=1)
y = data['Exited']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Save encoders and scaler for later use
with open('label_encoder_gender.pkl', 'wb') as file:
    pickle.dump(label_encoder_gender, file)

with open('onehot_encoder_geo.pkl', 'wb') as file:
    pickle.dump(onehot_encoder_geo, file)

with open('scaler.pkl', 'wb') as file:
    pickle.dump(scaler, file)

In [ ]:
def create_model(neurons=32,layers=1):
    model=Sequential()
    model.add(Dense(neurons,activation='relu',input_shape=(X_train.shape[1],)))

    for _ in range(layers-1):
        model.add(Dense(neurons,activation='relu'))

    model.add(Dense(1,activation='sigmoid'))
    model.compile(optimizer='adam',loss="binary_crossentropy",metrics=['accuracy'])

    return model



In [ ]:
model=KerasClassifier(layers=1,neurons=32,build_fn=create_model,verbose=1)

In [ ]:
param_grid = {
    'neurons': [16, 32, 64, 128],
    'layers': [1, 2],
    'epochs': [50, 100]
}

In [ ]:
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=3,verbose=1)
grid_result = grid.fit(X_train, y_train)

print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

Fitting 3 folds for each of 16 candidates, totalling 48 fits


c:\Users\prash\Customer-Churn-ANN-2\venv\Lib\site-packages\scikeras\wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)




Epoch 1/100


250/250 [==============================] - 1s 1ms/step - loss: 0.5643 - accuracy: 0.7268
Epoch 2/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4467 - accuracy: 0.8116
Epoch 3/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4181 - accuracy: 0.8181
Epoch 4/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4010 - accuracy: 0.8288
Epoch 5/100
250/250 [==============================] - 0s 1ms/step - loss: 0.3862 - accuracy: 0.8363
Epoch 6/100
250/250 [==============================] - 0s 1ms/step - loss: 0.3749 - accuracy: 0.8438
Epoch 7/100
250/250 [==============================] - 0s 1ms/step - loss: 0.3668 - accuracy: 0.8504
Epoch 8/100
250/250 [==============================] - 0s 1ms/step - loss: 0.3607 - accuracy: 0.8544
Epoch 9/100
250/250 [==============================] - 0s 1ms/step - loss: 0.3558 - accuracy: 0.8546
Epoch 10/100
250/250 [==============================] - 0s 1ms/step - loss: 0.3521 - ac